#### Задание

- Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества. — (1 балл)
- Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности. — (1 балл)
- Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT. — (1 балл)
- Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества клиентов.Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат. — (2 балла)
- Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишите отдельные запросы для минимальной и максимальной суммы. — (2 балла)
- Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций. — (1 балл)
- Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях) — (2 балла).

Описание таблицы customer: (Поле Описание)
- customer_id	id клиента
- first_name 	имя клиента
- last_name 	фамилия клиента
- gender 	пол
- DOB 	дата рождения
- job_title 	профессия
- job_industry_category	сфера деятельности
- wealth_segment 	сегмент благосостояния
- deceased_indicator 	флаг актуального клиента
- owns_car 	флаг наличия автомобиля
- address 	адрес проживания
- postcode 	почтовый индекс
- state 	штаты
- country 	страна проживания
- property_valuation	оценка имущества

Описание таблицы transaction: (Поле Описание)
- transaction_id	id транзакции
- product_id	id продукта
- customer_id	id клиента
- transaction_date	дата транзакции
- online_order	флаг онлайн-заказа
- order_status	статус транзакции
- brand	бренд
- product_line	линейка продуктов
- product_class	класс продукта
- product_size	размер продукта
- list_price	цена
- standard_cost	стандартная стоимость

In [ ]:
-- Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества.

select c.job_industry_category, count(*) as cust_count
from customer c 
group by c.job_industry_category;

![](1.png)

In [ ]:
-- Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности.

select date_part('month', to_date(t.transaction_date, 'dd.MM.yyyy')) as tran_month, c.job_industry_category as industry, sum(t.list_price) as tran_sum
from "transaction" t join customer c on t.customer_id = c.customer_id 
group by date_part('month', to_date(t.transaction_date, 'dd.MM.yyyy')), c.job_industry_category 
order by tran_month, industry;

![](2.png)

In [ ]:
--Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT.

select t.brand as brand, count(t.transaction_id) as tran_count
from "transaction" t join customer c on t.customer_id = c.customer_id 
where t.online_order = 'true' and t.order_status = 'Approved' and c.job_industry_category = 'IT'
group by t.brand
order by brand;

![](3.png)

In [ ]:
-- Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, 
-- отсортировав результат по убыванию суммы транзакций и количества транзакций.
-- Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат.

-- с помощью группировки
select c.customer_id as client
    ,sum(t.list_price) as ttl_price
    ,max(t.list_price) as max_price
    ,min(t.list_price) as min_price
    ,count(t.transaction_id) as tran_count  
from "transaction" t join customer c on t.customer_id = c.customer_id 
group by c.customer_id
order by ttl_price desc, tran_count desc;

-- с помощью оконных функций
select c.customer_id as client
    ,sum(t.list_price) over(partition by c.customer_id) as ttl_price
    ,max(t.list_price) over(partition by c.customer_id) as max_price
    ,min(t.list_price) over(partition by c.customer_id) as min_price
    ,count(t.transaction_id) over(partition by c.customer_id) as tran_count  
from "transaction" t join customer c on t.customer_id = c.customer_id 
order by ttl_price desc, tran_count desc;

- Вывод с помощью группировки содержит по одной строке для каждого клиента

![](4_1.png)


- Вывод с помощью оконных функций содержит дубли строк

![](4_2.png)

In [ ]:
-- Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). 
-- Напишите отдельные запросы для минимальной и максимальной суммы.

-- с наименьшей ценой
select distinct c.first_name, c.last_name
from "transaction" t join customer c on t.customer_id = c.customer_id
where t.list_price = (select min(list_price) from "transaction");

-- с наибольшей ценой
select distinct c.first_name, c.last_name
from "transaction" t join customer c on t.customer_id = c.customer_id
where t.list_price = (select max(list_price) from "transaction");

- с наименьшей

![](5_1.png)

- с наибольшей

![](5_2.png)

In [ ]:
-- Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций.

select distinct c.first_name, c.last_name
	,first_value(t.*) over(partition by c.customer_id order by to_date(t.transaction_date, 'dd.MM.yyyy')) as fst_row
from "transaction" t join customer c on t.customer_id = c.customer_id
order by c.first_name, c.last_name;

![](6.png)

In [ ]:
-- Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях)

with trans as (
	select c.first_name
		,c.last_name
		,c.job_title
		,to_date(t.transaction_date, 'dd.MM.yyyy') - 
			lag(to_date(t.transaction_date, 'dd.MM.yyyy')) 
				over(partition by c.customer_id order by to_date(t.transaction_date, 'dd.MM.yyyy')) as date_diff
	from "transaction" t join customer c on t.customer_id = c.customer_id
)

select *
from trans
where date_diff = (select max(date_diff) from trans);

![](7.png)